## 동적 웹 페이지 1

In [3]:
!pip install selenium

1. 크롬 브라우저에서 버전을 확인 한다. : 88.0.4324.190
2. https://chromedriver.chromium.org/downloads 에서 맞는 버전을 다운로드후 
   압축을 해제한 후 chromedriver.exe을 현재 디렉토리로 copy 한다.

In [12]:
from selenium import webdriver 

driver = webdriver.Chrome("chromedriver") 
driver.implicitly_wait(3)
driver.get("https://www.danawa.com/") 
                                           
login = driver.find_element_by_css_selector('li.my_page_service > a')
print("HTML 요소: ", login)
print("태그 이름: ", login.tag_name)
print("문자열: ", login.text)
print("href 속성: ", login.get_attribute('href'))

login.click()
driver.implicitly_wait(3)

my_id = "love1770"
my_pw = "ce30bfa8"

driver.find_element_by_id('danawa-member-login-input-id').send_keys(my_id)
driver.implicitly_wait(2)
driver.find_element_by_name('password').send_keys(my_pw)
driver.implicitly_wait(2)
driver.find_element_by_css_selector('button.btn_login').click()

HTML 요소:  <selenium.webdriver.remote.webelement.WebElement (session="afd5e9c76ebba2078a362990217c9905", element="26118f6d-05bb-45b9-90e4-70dfc15d2421")>
태그 이름:  a
문자열:  로그인
href 속성:  https://auth.danawa.com/login?url=http%3A%2F%2Fwww.danawa.com%2F


## 동적 웹 페이지 2

In [15]:
from selenium import webdriver 

driver = webdriver.Chrome("chromedriver") 
driver.implicitly_wait(3)
driver.get("https://www.danawa.com/") 

login = driver.find_element_by_css_selector('li.my_page_service > a')
login.click()
driver.implicitly_wait(3)

my_id = "love1770"
my_pw = "ce30bfa8"

driver.find_element_by_id('danawa-member-login-input-id').send_keys(my_id)
driver.implicitly_wait(2)
driver.find_element_by_name('password').send_keys(my_pw)
driver.implicitly_wait(2)
driver.find_element_by_css_selector('button.btn_login').click()
driver.implicitly_wait(2)

wishlist = driver.find_element_by_css_selector('li.interest_goods_service > a').click()
driver.implicitly_wait(10000)
html_src = driver.page_source

# print(html_src[:])

from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(html_src, 'html.parser')

wish_table = soup.select('table[class="tbl wish_tbl"]')[0]
wish_items = wish_table.select('tbody tr')

for item in wish_items:
    title = item.find('div', {'class':'tit'}).text
    price = item.find('span', {'class':'price'}).text
    link = item.find('a', href=re.compile('prod.danawa.com/info/')).get('href')
    print(title)
    print(price)
    print(link)
    print("\n")

driver.implicitly_wait(1000)
driver.close()

IndexError: list index out of range

## 한국은행 경제통계시스템 통계지표 활용하기 1

In [16]:
from selenium import webdriver 
import time

def download_bok_statistics():

    driver = webdriver.Chrome("chromedriver") 
    driver.implicitly_wait(3)
    driver.get("http://ecos.bok.or.kr/jsp/vis/keystat/#/key") 
    
    
    excel_download = driver.find_element_by_css_selector('img[alt="download"]')
    driver.implicitly_wait(3)
    
    excel_download.click()
    time.sleep(5)
    
    driver.close()
    print("파일 다운로드 실행...")

    return None

download_bok_statistics()

파일 다운로드 실행...


## 한국은행 경제통계시스템 통계지표 활용하기 2

In [17]:
from selenium import webdriver 
from bs4 import BeautifulSoup
import time

def download_bok_statistics_by_keyword():
    
    item_found = 0              
    while not item_found:
        
        keyword = ""       
        while len(keyword) == 0:
            keyword = str(input("검색할 항목을 입력하세요: "))
        
        driver = webdriver.Chrome("chromedriver")
        driver.implicitly_wait(3)
        driver.get("http://ecos.bok.or.kr/jsp/vis/keystat/#/key") 
        time.sleep(5)             
                   
        items1 = driver.find_elements_by_css_selector('a[class="ng-binding"]')
        items2 = driver.find_elements_by_css_selector('a[class="a-c1-list ng-binding"]')
        items3 = driver.find_elements_by_css_selector('a[class="a-c4-list ng-binding"]')
        driver.implicitly_wait(3)

        items = items1[1:] + items2 + items3    
        
        for idx, item in enumerate(items):
            if keyword in item.text:
                print("검색어 '%s'에 매칭되는 '%s' 통계지표를 검색 중..." % (keyword, item.text))
                item.click()
                item_found = 1
                time.sleep(5)
                break
            elif idx == (len(items) - 1):
                print("검색어 '%s'에 대한 통계지표가 존재하지 않습니다..." % keyword)
                driver.close()
                continue
            else:
                pass
            
    html_src = driver.page_source
    soup = BeautifulSoup(html_src, 'html.parser')
    driver.close()
  
    table_items = soup.find_all('td', {'class':'ng-binding'})
    date = [t.text for i, t in enumerate(table_items) if i % 3 == 0]
    value = [t.text for i, t in enumerate(table_items) if i % 3 == 1]    
    change = [t.text for i, t in enumerate(table_items) if i % 3 == 2]    
    
    result_file = open('bok_statistics_%s.csv' % keyword, 'w')
    
    for i in range(len(date)):
        result_file.write("%s, %s, %s" % (date[i], value[i], change[i]))
        result_file.write('\n')
    
    result_file.close()
    print("키워드 '%s'에 대한 통계지표를 저장하였습니다." % keyword)
    
    return date, value, change

result = download_bok_statistics_by_keyword()
print(result)

검색할 항목을 입력하세요: CD
검색어 'CD'에 매칭되는 'CD수익률(91일) ('21.03.10)' 통계지표를 검색 중...
키워드 'CD'에 대한 통계지표를 저장하였습니다.
(['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2020.8', '2020.9', '2020.10', '2020.11', '2020.12', '2021.1', '2021.2', '2021.3.3', '2021.3.4', '2021.3.5', '2021.3.8', '2021.3.9', '2021.3.10'], ['2.49', '1.76', '1.49', '1.44', '1.68', '1.69', '0.92', '0.68', '0.63', '0.63', '0.66', '0.66', '0.68', '0.73', '0.74', '0.74', '0.74', '0.74', '0.75', '0.75'], ['-0.23', '-0.73', '-0.27', '-0.05', '0.24', '0.01', '-0.77', '-0.11', '-0.05', '0.00', '0.03', '0.00', '0.02', '0.05', '0.00', '0.00', '0.00', '0.00', '0.01', '0.00'])
